# Gemini API Python quickstart

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/tutorials/quickstart_colab"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on Google AI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/quickstart_colab.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/tutorials/quickstart_colab.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Prerequisites

You can run the first part of this tutorial in Google Colab, which doesn't require additional environment configuration. The part when we send the email requires a regular IDE like VSCode.

Alternatively, to complete this quickstart locally, see the Python guidance in [Get started with the Gemini API](https://ai.google.dev/tutorials/quickstart). Also don't forget to run ```pip install -r requirements.txt```.

## Set up your API key

To use the Gemini API, you'll need an API key. If you don't already have one, create a key in Google AI Studio.

<a class="button" href="https://aistudio.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

To make it available to your program, add your API key in a json file which you can call ```config.json``` in the directory. Don't forget to add it to the ```.gitignore``` to avoid catastrophes.

In [34]:
# Import the Python SDK
import google.generativeai as genai # type: ignore
import json
import os

# Load the JSON file
config_file = "config.json"  # Replace with your JSON file's name if different
if os.path.exists(config_file):
    with open(config_file, "r") as file:
        config = json.load(file)
        GEMINI_API_KEY = config.get("GEMINI_API_KEY")
else:
    raise FileNotFoundError(f"Configuration file '{config_file}' not found.")

genai.configure(api_key=GEMINI_API_KEY)

## Initialize the Generative Model

Before you can make any API calls, you need to initialize the Generative Model.

In [35]:
model = genai.GenerativeModel('gemini-pro')

## Generate text

In [ ]:
example_prompt="Write a haiku about Generative AI and it's limitless potential."
print("Example Prompt:\n"+example_prompt,"\n")
response = model.generate_content(example_prompt)
print("Output:")
print(response.text)

## What's next

To learn more about working with the Gemini API, see the [Python tutorial](https://ai.google.dev/tutorials/python_quickstart).

If you're new to generative AI models, you might want to look at the
[concepts guide](https://ai.google.dev/docs/concepts) and the
[Gemini API overview](https://ai.google.dev/docs/gemini_api_overview).

# Phishing Prevention

### Generate Phishing Emails:
To generate a phishing email, we need to write a prompt. This prompt has to be creative in order to make it harder for people to guess that the email is a malicious one. This is a good article about it https://caniphish.com/blog/how-to-create-a-phishing-email.

Here is an example of how to build a prompt, here are the prompt parameters:





In [ ]:
name="John"
surname="Doe"
email="john.doe@example.com"
business_unit="Sales"
team_name="B2B Sales"

#Create an approximate combination of phishing parameters
phishing_examples = [
    {"Reason": "Account Suspicious Activity", "Fake Link": "https://example.com/secure-login", "Created By": "Sam Sussy"},
    {"Reason": "Password Expiry Notification", "Fake Link": "https://example.com/reset-password", "Created By": "Sally Sneaky"},
    {"Reason": "Exclusive Training Webinar", "Fake Link": "https://example.com/join-webinar", "Created By": "Richard Rascal"},
    {"Reason": "Email Storage Full", "Fake Link": "https://example.com/manage-storage", "Created By": "Bernard Bandit"}
]

We prompt Gemini for the body:

In [ ]:
#Take a random reason
import random
random_pick = random.choice(phishing_examples)

#Formulate a prompt
prompt=f'''Write an email from {random_pick["Created By"]} to {name} {surname} from the {team_name} team on the following theme: {random_pick["Reason"]}.
He will have to click on this link : {random_pick["Fake Link"]}. Only write the body of this email.'''

print("Prompt:",prompt)

In [ ]:
body = model.generate_content(prompt)
print(body.text)

We prompt Gemini for the subject, e.g. we feed it previous output and ask for the subject:

In [ ]:
subject = model.generate_content("Write me the subject of this email:\n"+body.text)
print(subject.text)

### Sending an email using your gmail account.

First, I would recommend you use a fresh gmail account to do what you have to do. So create yourself a dummy gmail account first. Try to keep it saved somewhere, you will probably be able to use it again someday.

Then go here: [Python quickstart to using Gmail](https://developers.google.com/gmail/api/quickstart/python)

Follow the things step by step and add the ```config.json```, and soon to be ```token.json``` to the ```.gitignore```

First we setup!

In [ ]:
from google.auth.transport.requests import Request # type: ignore
from google.oauth2.credentials import Credentials # type: ignore
from google_auth_oauthlib.flow import InstalledAppFlow # type: ignore
from googleapiclient.discovery import build # type: ignore
from googleapiclient.errors import HttpError # type: ignore

# If modifying these scopes, delete the file token.json.
SCOPES = [
    "https://www.googleapis.com/auth/gmail.send",
]

creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            "credentials.json", SCOPES
        )
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
        token.write(creds.to_json())

Then we create the necessary functions:

In [ ]:
import base64
import os.path
from email.mime.text import MIMEText

# This function has a weird return, ChatGPT made it that way, I didn't really look into it but here it is
# It just creates an email object
def create_message(sender, to, subject, message_text):
    """Create a message for an email."""
    message = MIMEText(message_text)
    message["to"] = to
    message["from"] = sender
    message["subject"] = subject
    return {"raw": base64.urlsafe_b64encode(message.as_bytes()).decode()}


def send_message(service, user_id, message):
    """Send an email message."""
    try:
        sent_message = (
            service.users()
            .messages()
            .send(userId=user_id, body=message)
            .execute()
        )
        print(f"Message sent! Message ID: {sent_message['id']}")
    except HttpError as error:
        print(f"An error occurred: {error}")

Then we send the email to ourselves!

In [ ]:
try:
    # Call the Gmail API
    service = build("gmail", "v1", credentials=creds)

    # Create and send a test email
    sender = "me"  # 'me' refers to the authenticated user
    recipient = "antoinegilliard.devemail@gmail.com"

    message = create_message(sender, recipient, subject.text, body.text)
    send_message(service, "me", message)

except HttpError as error:
    print(f"An error occurred: {error}")

The email needs to be sent, while being convincing and not just plain text. Have a look at this [interesting post](https://mailtrap.io/blog/python-send-html-email/).